In [2]:
pip install biopython

Note: you may need to restart the kernel to use updated packages.


In [8]:
from Bio import AlignIO

# Load MSA from a Clustal file
alignment = AlignIO.read("../data/AIntibody_COMPETITION_1.aligned.fasta", "fasta")

# Print alignment
print(alignment)

Alignment with 184171 rows and 449 columns
-------------------DIQMTQSPSSVSASVGDRVTITCRA...--- parental.0
-------------------DIQMTQSPSSVSASVGDRVTITCRA...--- phase1_h1_h2_am.1
-------------------DIQMTQSPSSVSASVGDRVTITCRA...--- phase1_h1_h2_am.10
-------------------DIQMTQSPSSVSASVGDRVTITCRA...--- phase1_h1_h2_am.100
-------------------DIQMTQSPSSVSASVGDRVTITCRA...--- phase1_h1_h2_am.1000
-------------------DIQMTQSPSSVSASVGDRVTITCRA...--- phase1_h1_h2_am.10000
-------------------DIQMTQSPSSVSASVGDRVTITCRA...--- phase1_h1_h2_am.10001
-------------------DIQMTQSPSSVSASVGDRVTITCRA...--- phase1_h1_h2_am.10002
-------------------DIQMTQSPSSVSASVGDRVTITCRA...--- phase1_h1_h2_am.10003
-------------------DIQMTQSPSSVSASVGDRVTITCRA...--- phase1_h1_h2_am.10004
-------------------DIQMTQSPSSVSASVGDRVTITCRA...--- phase1_h1_h2_am.10005
-------------------DIQMTQSPSSVSASVGDRVTITCRA...--- phase1_h1_h2_am.10006
-------------------DIQMTQSPSSVSASVGDRVTITCRA...--- phase1_h1_h2_am.10007
-------------------DIQMTQSPSS

In [66]:
from tqdm import tqdm

crds = ['QSIGTH', 'GASNLES', 'QQYKAYPLT', 'GYTFTSHA', 'ISPYRGDT']
consider_positions = []
for crd in crds:
    # get the index in the first sequence where the crd starts
    seq = alignment[0].seq
    
    candidate = ''

    # Find the first index where the crd starts
    start = None
    for i, aa in enumerate(seq):
        if aa == '-':
            continue
        sequence = seq[i:].replace('-', '')[:len(crd)]
        if str(sequence) == str(crd):
            start = i
            break


    if start is None:
        print(f"CRD {crd} not found in sequence")
        continue

    # Find the index where the crd ends
    rev_seq = seq[start:][::-1]
    rev_crd = crd[::-1]
    end = None
    for i, aa in enumerate(rev_seq):
        if aa == '-':
            continue
        sequence = rev_seq[i:].replace('-', '')[:len(rev_crd)]
        if str(sequence) == str(rev_crd):
            end = start + len(rev_seq) - i
            break

    print(f"CRD {crd} found from {start} to {end}", crd, seq[start:end])
    consider_positions.extend(range(start, end))

consider_positions

CRD QSIGTH found from 54 to 82 QSIGTH QS----I--------G-------T---H
CRD GASNLES found from 114 to 156 GASNLES G--------------------A---S-----N-------LES
CRD QQYKAYPLT found from 203 to 273 QQYKAYPLT QQ----Y----K------------------A--------Y----P-------------------L----T
CRD GYTFTSHA found from 319 to 337 GYTFTSHA GY---TF--T--S---HA
CRD ISPYRGDT found from 360 to 379 ISPYRGDT ISP--YRG------D---T


[54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 203,
 204,
 205,
 206,
 207,
 208,
 209,
 210,
 211,
 212,
 213,
 214,
 215,
 216,
 217,
 218,
 219,
 220,
 221,
 222,
 223,
 224,
 225,
 226,
 227,
 228,
 229,
 230,
 231,
 232,
 233,
 234,
 235,
 236,
 237,
 238,
 239,
 240,
 241,
 242,
 243,
 244,
 245,
 246,
 247,
 248,
 249,
 250,
 251,
 252,
 253,
 254,
 255,
 256,
 257,
 258,
 259,
 260,
 261,
 262,
 263,
 264,
 265,
 266,
 267,
 268,
 269,
 270,
 271,
 272,
 319,
 320,
 321,
 322,
 323,
 324,
 325,
 326,
 327,
 328,
 329,
 330,
 331,
 332,
 333,
 334,
 335,
 336,
 360,
 361,
 362,
 363,
 364,
 365,
 366,
 367,
 368,
 369,
 370,
 371,
 372,
 3

In [71]:
mutations_to_consider = []

# For each position get the frequency of each amino acid
for i in range(0, alignment.get_alignment_length()):
    # Get the column
    column = alignment[:, i]
    if i not in consider_positions:
        continue
    # Count the frequency of each amino acid
    frequency = {}
    for amino_acid in column:
        if amino_acid in frequency:
            frequency[amino_acid] += 1
        else:
            frequency[amino_acid] = 1
    frequency = {k: v / len(column) for k, v in frequency.items()}
    
    # Get most common amino acid
    most_common = max(frequency, key=frequency.get)
    if most_common == '-' and frequency[most_common] > 0.99:
        continue

    print(i, frequency)
    mutations_to_consider.append((i, frequency))


54 {'Q': 0.699784439461153, 'E': 0.09374983032073453, 'R': 0.11100553290148829, 'H': 0.07833480841174778, 'T': 0.0001466028853619734, 'X': 0.0002063299868057403, 'P': 0.004121169999619918, 'L': 0.003192685058994087, '-': 0.003757377654462429, 'D': 0.0008090307377382976, 'V': 0.00011945420288753387, 'K': 0.003350147417345836, 'F': 0.0003257841896932742, 'S': 0.00030406524371372255, 'A': 1.628920948466371e-05, 'G': 0.00030406524371372255, 'Y': 0.0003420733991779379, 'N': 0.00010859472989775806, 'I': 5.4297364948879035e-06, 'C': 1.628920948466371e-05}
55 {'S': 0.4430230600908938, 'N': 0.09379869794918852, 'D': 0.2059824836700675, 'T': 0.0627949025633786, 'A': 0.0307920356625093, 'H': 0.024786747099163276, 'G': 0.05270102241938199, 'R': 0.02061127973459448, 'I': 0.002910338761259916, 'Y': 0.01654983683641833, 'M': 0.0010805175624826926, 'P': 0.018917201948189456, 'E': 0.011179827442974192, 'V': 0.0060813048742744515, 'K': 0.005885834360458487, 'Q': 0.00013574341237219758, 'F': 0.0012434096

In [73]:
sum([len(v) for k, v in mutations_to_consider])

767